### 커피 특징
* !원두 이름!
    - 타입 (string)
    - 로스팅 포인트 (string)
    - 지속가능성 (string)
    - 컵 노트 (list of strings)
    - 바디감
    - 신맛
    - 단맛
    - 쓴맛

In [2]:
# package import
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import re
import time
# from tqdm import tqdm_notebook
from tqdm import tqdm

import pickle
import itertools

In [46]:
# # example dictionary into df
# dict = {'원두1':{'타입':'블랜드', '로스팅 포인트':'다크', '지속가능성':'-', '컵노트':['구운 아몬드', '밀크 초콜릿'], '바디감':4, '신맛':2, '단맛':4, '쓴맛':3}}
# # df = pd.DataFrame.from_dict(dict)
# # df.T

# # dict 추가
# dict['원두1']['테스트'] = '테스트'
### dict {'name':'원두1', ...}

In [4]:
# # Pickle dump the set of links to a file
# with open("links.pkl", "wb") as file:
#     pickle.dump(links, file)

NameError: name 'links' is not defined

In [3]:
# Load the set of links from the file
with open("links_559.pkl", "rb") as file:
    loaded_links = pickle.load(file)

# Print the length loaded set of links. Intended length : 556
print("Intended length : 559 == ", len(loaded_links))

# change set to list
links_list = list(loaded_links)
print("list length : ", len(links_list))

Intended length : 559 ==  559
list length :  559


In [ ]:
# loaded_links
# small_set = set(itertools.islice(loaded_links, 3))

***

### 제품 페이지에서의 데이터 수집

In [4]:
# return the level of features. (n/5)
def level_data(html):
    # Parse the HTML content
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div elements without the 'bg-opacity-30' class
    divs_without_opacity_30 = [div for div in soup.find_all('div', class_='bg-koke-green') if 'bg-opacity-30' not in div.get('class')]

    # Total number of blocks
    total_blocks = 5

    # Initialize position (n) to None
    position = None

    # Check if there are any elements without the 'bg-opacity-30' class
    if divs_without_opacity_30:
        # Get the index of the first element (0-based) and add 1 to get the position (1-based)
        position = divs_without_opacity_30[0].find_parent().find_all('div', class_='bg-koke-green').index(divs_without_opacity_30[0]) + 1

    #print(f"The element without 'bg-opacity-30' class is at position {position}/{total_blocks}")
    return position

# return type, roasting point, and sustainability of the coffee
def other(driver):
    type = ['블렌드', '싱글오리진', '디카페인']
    roast = ['라이트', '라이트미디엄', '미디엄', '미디엄다크', '다크']
    # sustain = ['직접무역', '공정무역', '유기농']
    tmp = {}
    
    elements = driver.find_elements(By.CSS_SELECTOR, 'li.lg\:block')
    
    result = []
    for i in range(len(elements)):
        html = elements[i].get_attribute('innerHTML')
        # Parse the HTML content
        soup = BeautifulSoup(html, 'html.parser')
    
        if soup.find_all('svg', 'hidden'):
            result.append(elements[i].text)
    
    for i in result:
        if i in type:
            tmp['type'] = i
        elif i in roast:
            tmp['roast'] = i
        else: # i in sustain:
            tmp['sustain'] = i
    return tmp

In [57]:
service = Service(path= './driver/chromedriver.exe')
driver = webdriver.Chrome(service=service)
time.sleep(3)

data = {}
for link in tqdm(links_list):
    driver.get(link)
    time.sleep(3)
    
    # product id
    pattern = r"https://biz.koke.kr/products/(\d+)"  # Regular expression pattern to match the product number
    match = re.search(pattern, link)
    if match:
        product_number = match.group(1)
        data[product_number] = dict.fromkeys(['coffee_name', '타입', '로스팅 포인트', '지속가능성', '컵 노트', '바디감', '신맛', '단맛', '쓴맛'])  # add coffee names as keys and empty dictionary as values
    
    # coffee_name
    raw = driver.find_elements(By.CSS_SELECTOR,"div.flex.flex-col")
    coffee_name = raw[0].find_element(By.TAG_NAME,'h1').text
    data[product_number]['coffee_name'] = coffee_name
    time.sleep(3)
    
    # add data : '바디감', '신맛', '단맛', '쓴맛'
    temp = driver.find_elements(By.CSS_SELECTOR,'div.flex.justify-between.w-full.h-2\.5.overflow-hidden.rounded-full') # len(temp) = 4
    keys = ['바디감', '신맛', '단맛', '쓴맛']
    for i in range(4):
        html = temp[i].get_attribute('innerHTML')
        data[product_number][keys[i]] = level_data(html)
    time.sleep(3)
    
    # others : '타입', '로스팅 포인트', '지속가능성'
    tmp = other(driver)
    for i in range(len(tmp)):
        keys = list(tmp.keys())
        if keys[i] == 'type':
            data[product_number]['타입'] = tmp['type']
        elif keys[i] == 'roast':
            data[product_number]['로스팅 포인트'] = tmp['roast']
        else:
            data[product_number]['지속가능성'] = tmp['sustain']
    time.sleep(3)
            
    # 컵 노트
    try:
        notes = driver.find_element(By.CSS_SELECTOR, 'li.tracking-wide.koke-word-keep-all').text
        data[product_number]['컵 노트'] = notes
    except:
      data[product_number]['컵 노트'] = '-'
      
    time.sleep(1)       
    
driver.quit()

time.sleep(5)

if len(data.keys()) == 559:
    print('{} out of 559'.format(len(data.keys())), 'Success!')
    df = pd.DataFrame.from_dict(data)
    newdf = df.T
    newdf.to_csv('data_fin.csv')
else:
    print('{} out of 559'.format(len(data.keys())))
    # export current collected data
    comparedf = pd.DataFrame.from_dict(data)
    comparedf = comparedf.T
    comparedf.to_csv('compare.csv')

100%|██████████| 559/559 [2:14:44<00:00, 14.46s/it]  


559 out of 559 Success!


In [39]:
############ print product id from the pickle ############

# Regular expression pattern to match the product number
pattern = r"https://biz.koke.kr/products/(\d+)"

# Initialize an empty list to store the extracted product numbers
product_numbers = []

# Iterate through the links and extract the product numbers
for link in links_list:
    match = re.search(pattern, link)
    if match:
        product_number = match.group(1)  # Extract the first captured group (product number)
        product_numbers.append(int(product_number))
        
print(len(product_numbers))
print('examples', product_numbers[:5])

559
examples [1084, 4699, 1727, 3072, 3217]


In [18]:
######## get collected data's ids ########
comparedf =  pd.read_csv('compare.csv')
comparedf.rename(columns = {'Unnamed: 0' : 'coffee_name'}, inplace = True)
df_id = comparedf['ID']
id_ls = list(df_id)

In [47]:
######## compare current data size and intended size ########
print('size of collected ids from data: ',len(df_id), '&', 'example : ', id_ls[:5])
print('intended size: ',len(product_numbers), '&', 'example : ', product_numbers[:5])

size of collected ids from data:  540 & example :  [1084, 4699, 1727, 3072, 3217]
intended size:  559 & example :  [1084, 4699, 1727, 3072, 3217]


In [48]:
######## 540 out of 559 => missing data == 19 ########
def non_match_elements(list_a, list_b):
    non_match = []
    for i in list_a:
        if i not in list_b:
            non_match.append(i)
    return non_match

missing_list = non_match_elements(product_numbers, id_ls)
len(missing_list)

19

In [50]:
missing_links = []
pattern = "https://biz.koke.kr/products/"
for i in missing_list:
    link = pattern + str(i)
    missing_links.append(link)
print('size of missing link :', len(missing_links), 'example :', missing_links[:2])

size of missing link : 19 example : ['https://biz.koke.kr/products/3214', 'https://biz.koke.kr/products/4737']


***

In [ ]:
########## Export data as csv file ##########
df = pd.DataFrame.from_dict(data)
newdf = df.T
newdf.to_csv('data.csv')

In [29]:
########## read created csv file ##########
tab = pd.read_csv('data.csv')
tab

In [30]:
tab

,Unnamed: 0,타입,로스팅 포인트,지속가능성,컵 노트,바디감,신맛,단맛,쓴맛
0,과테말라 디카페인,디카페인,미디엄다크,NaN,-,4,1,3,2
1,브라질 디카페인,디카페인,미디엄다크,NaN,-,3,1,4,3
2,77도 블렌드,블렌드,미디엄다크,NaN,<selenium.webdriver.remote.webelement.WebEleme...,4,1,4,3
3,밤의 유영,디카페인,미디엄다크,NaN,-,3,2,3,3
4,올굳 블렌드,블렌드,미디엄,NaN,-,3,3,4,3
...,...,...,...,...,...,...,...,...,...
532,발렌타인 플러스,블렌드,미디엄다크,NaN,<selenium.webdriver.remote.webelement.WebEleme...,5,2,3,2
533,과테말라 아구아 티비아 게이샤 워시드,싱글오리진,라이트,NaN,-,3,3,5,1
534,Blend N.4,블렌드,다크,NaN,<selenium.webdriver.remote.webelement.WebEleme...,4,1,4,3
535,닻별,블렌드,다크,유기농,-,4,2,4,4


***

In [35]:
service = Service(path= './driver/chromedriver.exe')
driver = webdriver.Chrome(service=service)

test = {}
for link in tqdm(links_list[:5]):
    name = {}
    driver.get(link)
    time.sleep(3)
    
    # name
    raw = driver.find_elements(By.CSS_SELECTOR,"div.flex.flex-col")
    coffee_name = raw[0].find_element(By.TAG_NAME,'h1').text
    test[coffee_name] =  dict.fromkeys(['ID', '타입', '로스팅 포인트', '지속가능성', '컵 노트', '바디감', '신맛', '단맛', '쓴맛'])  # add coffee names as keys and empty dictionary as values
    time.sleep(3)
    
    # product id
    pattern = r"https://biz.koke.kr/products/(\d+)"  # Regular expression pattern to match the product number
    match = re.search(pattern, link)
    if match:
        product_number = match.group(1)
        test[coffee_name]['ID'] = product_number
        
driver.quit()

100%|██████████| 5/5 [00:35<00:00,  7.08s/it]


In [39]:
testdf = pd.DataFrame(test)
testdf.T

,ID,타입,로스팅 포인트,지속가능성,컵 노트,바디감,신맛,단맛,쓴맛
과테말라 디카페인,3394,None,None,None,None,None,None,None,None
브라질 디카페인,4824,None,None,None,None,None,None,None,None
77도 블렌드,217,None,None,None,None,None,None,None,None
밤의 유영,4328,None,None,None,None,None,None,None,None
올굳 블렌드,4676,None,None,None,None,None,None,None,None
